# Model Exploration and Manipulation with `py_dss_toolkit`: A Hands-on Example

This notebook focuses on the usage of the model functionality within the py_dss_toolkit package. It demonstrates how to explore the various components of a model, including retrieving key data about buses, lines, and loads. You'll also learn how to manipulate the model by adding new elements, modifying existing elements, and disabling components such as loads. This example highlights how py_dss_toolkit provides a streamlined interface for interacting with your models, making it easier to explore and adjust them.

In [1]:
import py_dss_interface
from py_dss_toolkit import dss_tools
import ipynbname

Define the path to the DSS file

In [2]:
import ipynbname
nb_path = ipynbname.path()
dss_file = nb_path.parent.parent.joinpath("feeders", "123bus", "IEEE123Master.dss")

Create an instance of DSS

In [3]:
dss = py_dss_interface.DSS()

Connect DSS instance to the dss_tools

In [4]:
dss_tools.update_dss(dss)

Compile the DSS model from the specified file, load bus coordinates for visualization, and solve the power flow for the system

In [5]:
dss.text(f"compile [{dss_file}]")
dss.text(f"buscoords buscoords.dat")
dss.text(f"solve")

''

### Retrieve some examples of dataframes from the model

In [6]:
summary_df = dss_tools.model.summary_df  # Summary of the model
summary_df

,count
buses,132.000000
nodes,278.000000
ckt elements,237.000000
vsource,1.000000
transformer,8.000000
regcontrol,7.000000
line,126.000000
capacitor,4.000000
load,91.000000
line length,38.983000


In [7]:
segments_df = dss_tools.model.segments_df  # Segments data of the network
segments_df.head()

,name,bus1,bus2,type,x1,y1,x2,y2
0,transformer.reg1a,150,150r,transformer,100.0,1500.0,100.0,1500.0
1,line.l115,149,1,line,300.0,1500.0,700.0,1500.0
2,line.l1,1,2,line,700.0,1500.0,700.0,1675.0
3,line.l2,1,3,line,700.0,1500.0,700.0,1250.0
4,line.l3,1,7,line,700.0,1500.0,1000.0,1500.0


In [8]:
buses_df = dss_tools.model.buses_df  # Bus data
buses_df.head()

,name,nodes,num_nodes,kv_base,distance,coord_defined,x,y,latitude,longitude,all_pce_active_bus,all_pde_active_bus,line_list,line_total_miles,load_list,section_id,total_customers,unique_node_number
0,150,"[1, 2, 3]",3,2.401777,0.0,1,100.0,1500.0,0.0,0.0,[Vsource.source],[Transformer.reg1a],[None],0.0,[None],0,0,4
1,150r,"[1, 2, 3]",3,2.401777,0.0,1,100.0,1500.0,0.0,0.0,[None],"[Line.sw1, Transformer.reg1a]",[LINE.sw1],0.0,[None],0,0,4
2,149,"[1, 2, 3]",3,2.401777,0.0,1,300.0,1500.0,0.0,0.0,[None],"[Line.l115, Line.sw1]","[LINE.l115, LINE.sw1]",0.0,[None],0,0,4
3,1,"[1, 2, 3]",3,2.401777,0.0,1,700.0,1500.0,0.0,0.0,[Load.s1a],"[Line.l115, Line.l1, Line.l2, Line.l3]","[LINE.l115, LINE.l1, LINE.l2, LINE.l3]",0.0,[LOAD.s1a],0,0,4
4,2,[2],1,2.401777,0.0,1,700.0,1675.0,0.0,0.0,[Load.s2b],[Line.l1],[LINE.l1],0.0,[LOAD.s2b],0,0,1


In [9]:
lines_df = dss_tools.model.lines_df  # Line data
lines_df.head()

,name,bus1,bus2,linecode,length,phases,r1,x1,r0,x0,...,heightunit,conductors,normamps,emergamps,faultrate,pctperm,repair,basefreq,enabled,like
0,l115,149,1,1,0.4,3,----,----,----,----,...,m,,400,600,0.1,20,3,60,true,
1,l1,1.2,2.2,10,0.175,1,----,----,----,----,...,m,,400,600,0.1,20,3,60,true,
2,l2,1.3,3.3,11,0.25,1,----,----,----,----,...,m,,400,600,0.1,20,3,60,true,
3,l3,1.1.2.3,7.1.2.3,1,0.3,3,----,----,----,----,...,m,,400,600,0.1,20,3,60,true,
4,l4,3.3,4.3,11,0.2,1,----,----,----,----,...,m,,400,600,0.1,20,3,60,true,


### Explore some of the methods of the model
Add a line in series with the Vsource, also adding a meter at the source

In [10]:
dss_tools.model.add_line_in_vsource(add_meter=True)

Retrieve the data of the newly added line and inspect the first bus in the system

In [11]:
new_line = dss_tools.model.lines_df.iloc[-1]
new_bus = dss_tools.model.buses_df.iloc[0]

# Display the new line and bus
new_line, new_bus

(name                                                  feeder_head
 bus1                                                150_unrealbus
 bus2                                                          150
 linecode                                                         
 length                                                      0.001
 phases                                                          3
 r1                                                              1
 x1                                                              1
 r0                                                              1
 x0                                                              1
 c1                                                            1.1
 c0                                                              1
 rmatrix                                          [1 |0 1 |0 0 1 ]
 xmatrix                                          [1 |0 1 |0 0 1 ]
 cmatrix         [1.066667 |-0.03333333 1.066667 |-0.03333333 

Disable all load elements in the system

In [12]:
dss_tools.model.disable_elements_type("load")

# Check the updated load data
loads_df = dss_tools.model.loads_df
loads_df

,name,phases,bus1,kv,kw,pf,model,yearly,daily,duty,...,zipv,%seriesrl,relweight,vlowpu,puxharm,xrharm,spectrum,basefreq,enabled,like


Fetch data for a specific line (l115) and inspect its properties

In [13]:
line_l115_data = dss_tools.model.element_data("line", "l115")
line_l115_data

name,l115
bus1,149
bus2,1
linecode,1
length,0.4
phases,3
r1,----
x1,----
r0,----
x0,----
c1,----


Check if the element 'l115' is present in the model

In [14]:
is_line_l115 = dss_tools.model.is_element_in_model("line", "l115")
is_line_l115

True

Modify properties of the line 'l115' (e.g., change its rating) and verify the changes made to the line 'l115'

In [15]:
dss_tools.model.edit_element("line", "l115", dict(normamps=800, emergamps=1000))
new_line_l115_data = dss_tools.model.element_data("line", "l115")
new_line_l115_data

name,l115
bus1,149
bus2,1
linecode,1
length,0.4
phases,3
r1,----
x1,----
r0,----
x0,----
c1,----


Add a new load element to the system and check the updated load data

In [16]:
dss_tools.model.add_element("load", "my_load", dict(phases=3, bus1=1, kv=4.16, kw=100))
new_loads_df = dss_tools.model.loads_df
new_loads_df

,name,phases,bus1,kv,kw,pf,model,yearly,daily,duty,...,zipv,%seriesrl,relweight,vlowpu,puxharm,xrharm,spectrum,basefreq,enabled,like
0,my_load,3,1,4.16,100,0.88,1,,,,...,,50,1,0.5,0,6,defaultload,60,true,


### Conclusion

In this notebook, we demonstrated several key capabilities of the model functionality within py_dss_toolkit. By exploring and manipulating the model, we showcased how this tool can be used to interact with a test case feeder. This includes adding new elements, modifying existing components, and disabling elements like loads. These examples illustrate the flexibility of the tool's model capabilities. If you have any suggestions or ideas for additional features that would enhance the functionality, please feel free to reach out to me. I'm always open to feedback and collaboration to improve this tool.